Asociación de Palabras

In [4]:
from nltk.corpus import XMLCorpusReader

def leer_corpus():
    corpus_root = "ap/"
    wiki = XMLCorpusReader(corpus_root ,'output.xml')
    print(wiki())

leer_corpus()

TypeError: 'XMLCorpusReader' object is not callable

In [35]:
from xml.dom import minidom
xmldoc = minidom.parse('ap/ap.xml')
doclist = xmldoc.getElementsByTagName('DOC')

docs = {}

for doc in doclist:
    docno = doc.getElementsByTagName('DOCNO')[0].firstChild.data.strip()
    text = doc.getElementsByTagName('TEXT')[0].firstChild.data.strip()
    docs[docno] = text

In [52]:
from nltk.corpus import wordnet as wn

def is_noun(tag):
    return tag in ['NN', 'NNS', 'NNP', 'NNPS']

def is_verb(tag):
    return tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

def is_adverb(tag):
    return tag in ['RB', 'RBR', 'RBS']

def is_adjective(tag):
    return tag in ['JJ', 'JJR', 'JJS']

def penn_to_wn(tag):
    if is_adjective(tag):
        return wn.ADJ
    elif is_noun(tag):
        return wn.NOUN
    elif is_adverb(tag):
        return wn.ADV
    elif is_verb(tag):
        return wn.VERB
    return None

In [79]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords

from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

def lemmatize_with_tag(word, tag):
    wordType = penn_to_wn(tag)
    if wordType is None:
        return None
    return wordnet_lemmatizer.lemmatize(word, pos=wordType)

vocabulary = {}

for docno, text in docs.items():
    sents = sent_detector.tokenize(text)
    for sent in sents:
        # tokens = nltk.word_tokenize(sent)
        tokens = tokenizer.tokenize(sent)
        token_with_tags = nltk.pos_tag(tokens)
        for token_with_tag in token_with_tags:
            word = lemmatize_with_tag(token_with_tag[0], token_with_tag[1])
            if word is not None and word not in stopwords.words('english'):
                if word in vocabulary:
                    vocabulary[word] += 1
                else:
                    vocabulary[word] = 1

1) Separar en frases
2) Lematizar
3) Stop words
4) Tokenize (por doc)
5) frec (de nltk)

In [80]:
print(len(vocabulary))

37133


In [81]:
import operator
sorted_vocabulary = sorted(vocabulary.items(), key=operator.itemgetter(1))
print(sorted_vocabulary)

[(u'Szczecin', 1), (u'mid-week', 1), (u'Nampa', 1), (u'five-nation', 1), (u'regularize', 1), (u'Semiconductor-NEC', 1), (u'Sedaka', 1), (u'Saca', 1), (u'Pergamon', 1), (u'steadfastness', 1), (u'all-news', 1), (u'Shocked', 1), (u'feasibility', 1), (u'Gorbachebroad', 1), (u'sustaining', 1), (u'inanimate', 1), (u'Elekta', 1), (u'smoker-cookers', 1), (u'numeral', 1), (u'presentence', 1), (u'Orderly', 1), (u'china', 1), (u'chink', 1), (u'quart', 1), (u'Ginger', 1), (u'topography', 1), (u'170-bed', 1), (u'Kielce', 1), (u'Zhirinovski', 1), (u'cannibal', 1), (u'Laboda', 1), (u'steelmaking', 1), (u'down-to-earth', 1), (u'unpack', 1), (u'440,005-', 1), (u'six-piece', 1), (u'Sugawara', 1), (u'_Smith', 1), (u'caporegime', 1), (u'wand', 1), (u'Lt', 1), (u'cookery', 1), (u'Receiving', 1), (u'Attention', 1), (u'_Prudential-Bache', 1), (u'Nussbaum', 1), (u'Falcon', 1), (u'Correspondent', 1), (u'Giusto', 1), (u'Gusts', 1), (u'pigment', 1), (u'Production-Philippines', 1), (u'colorfully', 1), (u'Hoping',

In [74]:
from nltk.collocations import BigramCollocationFinder
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(vocabulary.keys())
finder.nbest(bigram_measures.pmi, 10)

[(u"''Journalists", u'Giancarlo'),
 (u"'46", u'29-30'),
 (u"'50s", u'bone-dry'),
 (u"'60s", u'comptroller'),
 (u"'70s", u'Ninety-two'),
 (u"'80s", u'Lydia'),
 (u"'83", u'Fresh'),
 (u"'84", u'charm'),
 (u"'88", u'preconvention'),
 (u"'90s", u'Actor-comedian')]